# Save IGRINS observation times to a machine readable log

We'll need this to register the TESS observations.

### Get the IGRINS epoch times.

In [ ]:
import glob

In [ ]:
from astropy.io import fits

In [ ]:
reduced_fns = glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/*/reduced/SDCH*spec_a0v.fits')

In [ ]:
raw_fns = glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/*/SDCH*.fits')

In [ ]:
len(reduced_fns)

In [ ]:
len(raw_fns)

In [ ]:
hdu = fits.open(raw_fns[23])[0]
hdr = hdu.header

In [ ]:
keep_keys = ['OBJECT', 'EXPTIME', 'FRMTYPE', 'OBJTYPE', 'JD-OBS']

In [ ]:
import pandas as pd

In [ ]:
df_raw = pd.DataFrame()

for i, fn in enumerate(raw_fns):
    try:
        hdu = fits.open(fn)[0]
        hdr = hdu.header
        out_dict = {key:hdr[key] for key in keep_keys}
        out_dict['night_dir'] = fn[44:44+8]
        df = pd.DataFrame(out_dict, index=[i])
        df_raw = df_raw.append(df)
    except:
        print(f"{fn} had a problem")

In [ ]:
df_log = df_raw.sort_values('JD-OBS').reset_index(drop=True)

Make a quick summary of how many spectra were acquired on each night:

In [ ]:
from astropy.time import Time

In [ ]:
df_log['BTJD'] = df_log['JD-OBS']- 2457000.0

In [ ]:
df_out = df_log.groupby(['night_dir', 'OBJECT', 'EXPTIME']).BTJD.mean().round(decimals=4).to_frame()
df_out

Neat! Let's save these so we can read them in later.

In [ ]:
df_out.to_csv('../../data/IGRINS/2021A_metadata_log.csv')

In [ ]:
! head ../../data/IGRINS/2021A_metadata_log.csv

Great!  Now we can take those dates and overlay them on TESS lightcurves.